# Proximal Policy Optimisation on LunarLanderContinuous-v3

## Introduction

## Part 1: Theoretical Backgrounds

A common estimator for policy gradient at time step $t$ is defined as:

$$
\begin{align*}
	\nabla_\theta \hat{J(\theta)} = \hat {\mathbb E}_t\left[ \nabla_\theta\log \pi_\theta(a_t\mid s_t) \hat A_t \right]
\end{align*}
$$

where $\hat A_t$ is an estimator of the advantage function at time step $t$. (Please refer to the [A2C Notebook](../a2c/lunar-lander-a2c.ipynb) for deriving the above estimator).

In this case, the estimator is obtained by differentiating the objective function

$$
J(\theta) = \hat {\mathbb E}_t\left[ \log \pi_\theta(a_t\mid s_t)\hat A_t \right].
$$

One of the drawbacks of the above method, it leads to "destructively large" policy updates, making our gradient estimates have high variance, meaning that the gradient estimates can fluctuate significantly from one batch of samples to another. This leads to the update of the parameters $\theta$ erratic and unstable.

Hence TRPO is suggested ([Schulman, 2015]()). The objective is maximised subject to a constraint on the size of the policy update:

$$
\begin{gather*}
\text{maximise}_\theta& \hat {\mathbb E}_t\left[ \frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_{old}}(a_t\mid s_t)} \hat A_t \right]
\\
\text{subject to} & \hat {\mathbb E}_t\left[ KL\left[ \pi_{\theta_{old}}(\cdot\mid s_t), \pi_\theta(\cdot\mid s_t) \right] \right] \leq \delta
\end{gather*}
$$

for some hyperparameter $\delta$. It used probability ratio, which compares the probability of taking an action under the new and the old policy. Now it becomes an optimisation problem under under a constraint, which prevents the new policy to deviate too much from an old policy, contributing to more stable learning. If $\hat A_t$ is positive, we want to increase the probability of taking $a_t$ in state $s_t$. The objective function $\hat{\mathbb E}_t\left[ \frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_{old}}(a_t\mid s_t)} \hat A_t \right]$ achieves this by making the probability ratio greater than 1, which increases the overall value of the expectation. On the other hand, if $\hat A_t$ is negative, we want to minimise the absolute value of our estimate of expectation, which is done by making the probability ratio less than 1.

Although this setting makes the overall learning process quite robust, it leaves a room for improvement. The biggest problem of TRPO is that it is inefficient. Calculating the KL divergence between the new and the old policies requires second-order optimisation techniques such as conjugate gradient and Hessian-vector products, which can be computationally expensive. PPO ([Schulman, 2017]()) overcomes this issue by introducing a clipped surrogate objective:

$$
L^{CLIP}(\theta) = \min\left( r_t(\theta) \hat A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat A_t \right)
$$

where $\epsilon$ is a hyperparameter and $r_t(\theta)=\frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_{old}}(a_t\mid s_t)}$ is the probabiltiy ratio. Trivially, we have $r_t(\theta_{old})=1$. Note that the constraint is directly integrated in the form of "clip" function. That is, whenever the probability ratio is greater or less than our pre-specified threshold, we "clip" and force the value to be that threshold. It is much simpler to implement, as it only requires first-order optimisation methods.

The objective function in PPO is:

$$
L_t^{CLIP+VF+S}(\theta) = \hat{\mathbb E}_t \left[ L_t^{CLIP}(\theta) - c_1 L_t^{VF} + c_2 S[\pi_\theta](s_t) \right]
$$

where $c_1$, $c_2$ are coefficients, $S$  is the entropy bonus, and $L_t^{VF}$ is a MSE loss of the value function. We already covered what $L_t^{CLIP}$, so now we figure out what the other two terms are.

Note that we have an advantage estimate $\hat A_t$ in $L_t^{CLIP}$. This advantage estimate ([Minh et al., 2016]()) is defined as

$$
\hat A_t = \delta_t + (\gamma\lambda)\delta_{t+1} + \cdots + (\gamma \lambda)^{T-t+1}\delta_{T-1}
$$

where $\delta_t = r_t+\gamma V(s_{t+1}) - V(s_t)$. To estimate the advantage function, the value functon is crucial. A more accurate value function leads to better advantage estimates and, consequently, better policy updates. Therefore, we train the value function alongside the policy. 

The last term, $S[\pi_\theta](s_t)$ represents the entropy ([Williams, 1992]()) of the policy $\pi_\theta$ at state $s_t$. Defined as

$$
S[\pi_\theta](s_t) 
=
\begin{cases}
-\int_{a\in\mathcal A}\pi_\theta(a\mid s_t) \log_2 \pi_\theta(a\mid s_t) da & \mathcal A \text{ is continuous}
\\
-\sum_{a\in\mathcal A}\pi_\theta(a\mid s_t) \log_2 \pi_\theta(a\mid s_t) & \mathcal A \text{ is discrete}
\end{cases}
$$

entropy quantifies how unpredictable a policy's actions are. By adding an entropy bonus to the loss function, we incentivise the agent to explore a wide range of actions, helping prevent the policy from converging too quickly to a suboptimal policy.


![ppo-algorithm](../../figures/ppo.png)

## Part 2: Implementation

### Import libraries

In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tqdm
import random
from collections import deque, namedtuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

### Define actor and critic networks

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, action_dim)
        
		self.log_stds = 

In [ ]:
class Critic(nn.Module):
	def __init__(self, state_dim, action_dim, hidden_dim=64):
		super(Critic, self).__init__()
		self.fc1 = nn.Linear(state_dim, hidden_dim)
		self.fc2 = nn.Linear(hidden_dim, hidden_dim)
		self.fc3 = nn.Linear(hidden_dim, action_dim)

	def forward(self, state, action):
		x = F.relu(self.fc1(state))
		x = F.relu(self.fc2(torch.cat([x, action], dim=1)))
		x = self.fc3(x)
		return x

### Define replay buffer

In [ ]:
class ReplayBuffer():
	def __init__(self, action_dim, memory_size, batch_size):
		self.action_dim = action_dim
		self.memory_size = memory_size
		self.batch_size = batch_size
		self.memory = deque(maxlen=memory_size)
		self.experience = namedtuple(
							"Experience", 
				   			field_names=["state", "action", "reward", "next_state", "done", "log_prob", "val"])

	def __len__(self):
		return len(self.memory)
	
	def push(self, state, action, reward, next_state, done, log_prob, val):
		e = self.experience(state, action, reward, next_state, done, log_prob, val)
		self.memory.append(e)
	
	def sample(self):
		experiences = random.sample(self.memory, k=self.batch_size)
		states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
		actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
		rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
		next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
		dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
		log_probs = torch.from_numpy(np.vstack([e.log_prob for e in experiences if e is not None])).float().to(device)
		vals = torch.from_numpy(np.vstack([e.val for e in experiences if e is not None])).float().to(device)
		
		return (states, actions, rewards, next_states, dones, log_probs, vals)

### Define PPO agent

In [ ]:
class PPO():
	def __init__(self,):
		pass
	
	def act(self, state):
		pass
	
	def save_to_memory(self,):
		pass

	def optimize(self,):
		pass

        


### Train agent

In [ ]:
# Hyperparameters

train_env = gym.make("LunarLanderContinuous-v3")
state_dim = train_env.observation_space.shape[0]
action_dim = train_env.action_space.shape[0]
target_score = 250
gamma = 0.99
lr = 1e-3
n_episodes = 5000
batch_size = 64
memory_size = T


In [ ]:
agent = PPO(state)

In [ ]:
def train(env, agent, n_episodes, max_steps, gamma, target):
	score_history = []
	
	bar_format = '{l_bar}{bar:10}| {n:4}/{total_fmt} [{elapsed:>7}<{remaining:>7}, {rate_fmt}{postfix}]'
	progress_bar = tqdm.trange(n_episodes, unit="ep", bar_format=bar_format, ascii=True)

	for epsode in progress_bar:
		state, _ = env.reset()

		score = 0
		
		for t in range(max_steps):
			# Run policy for T timesteps
			action = agent.act(state)
			next_state, reward, done, _ = env.step(action)
			log_prob =
			if done:
				val = 0
				break
			val = 
			agent.save_to_memory(state, action, reward, next_state, done, log_prob, val)
			
			# Compute advantage estimates (A_1, ..., A_t)
			

		# Opitmise surrogate L w.r.t. theta, with K epochs and minibatch size M <= T
		agent.optimize()
		






In [ ]:
# Initialise actor, critic, and replay buffer

Here actor network will output action probabilities and critc network will output Q value for a given state(We don't need to pass action and state to critc network since we found above that calculating advanatge depends up on states).

Data store is the memory where we store our state,action,action probability and rewards. It is also able to generate batches of data for traning.

For each time step in the episode we store current_state, action, action probability, Q value(output from critic network for that state), reward and done status in the Datastore(memory)

 Now when we reach required number of steps in the episode, we will train our model.With already generated batches we will start our traning for n_epoch. For each epoch we will do the following:

Calculate the advantage array using the reward array and Q value array (we are saying array because we are having a batch of data).

### Visualisation

### Test